In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need these
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/text_classification/imdb/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install tf-nightly-2.0-preview

In [0]:
import tensorflow as tf
import numpy as np
import pprint
import logging
import time

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0-dev20190324
GPU Enabled: False


In [0]:
def data_generator(f_path, params):
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      line = line.rstrip()
      label, text = line.split('\t')
      text = text.split(' ')
      x = [params['word2idx'].get(w, len(word2idx)) for w in text]
      if len(x) > params['max_len']:
        x = x[:params['max_len']]
      y = int(label)
      yield x, y


def dataset(is_training, params):
  _shapes = ([None], ())
  _types = (tf.int32, tf.int32)
  _pads = (0, -1)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['num_samples'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
class KernelAttentivePooling(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.kernel = tf.keras.layers.Dense(units=1,
                                        activation=tf.tanh,
                                        use_bias=False)

  
  def call(self, inputs, training=False):
    x, masks = inputs
    # alignment
    align = tf.squeeze(self.kernel(self.dropout(x, training=training)), -1)
    # masking
    paddings = tf.fill(tf.shape(align), float('-inf'))
    align = tf.where(tf.equal(masks, 0), paddings, align)
    # probability
    align = tf.nn.softmax(align)
    align = tf.expand_dims(align, -1)
    # weighted sum
    return tf.squeeze(tf.matmul(x, align, transpose_a=True), -1)

In [0]:
class Model(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    
    self.embedding = tf.Variable(np.load('../vocab/word.npy'),
                                 dtype=tf.float32,
                                 name='pretrained_embedding')
    
    self.attentive_pooling = KernelAttentivePooling(params)
    
    self.out_linear = tf.keras.layers.Dense(2)
  
  
  def call(self, inputs, training=False):
    if inputs.dtype != tf.int32:
      inputs = tf.cast(inputs, tf.int32)
    masks = tf.sign(inputs)
    
    x = tf.nn.embedding_lookup(self.embedding, inputs)
    
    x = self.attentive_pooling((x, masks), training=training)
    
    x = self.out_linear(x)
    
    return x

In [0]:
params = {
  'vocab_path': '../vocab/word.txt',
  'train_path': '../data/train.txt',
  'test_path': '../data/test.txt',
  'num_samples': 25000,
  'num_labels': 2,
  'batch_size': 32,
  'max_len': 1000,
  'dropout_rate': 0.2,
  'kernel_size': 5,
  'num_patience': 3,
  'lr': 3e-4,
}

In [0]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True    

In [0]:
word2idx = {}
with open(params['vocab_path']) as f:
  for i, line in enumerate(f):
    line = line.rstrip()
    word2idx[line] = i
params['word2idx'] = word2idx
params['vocab_size'] = len(word2idx) + 1


model = Model(params)
model.build(input_shape=(None, None))
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.95)
optim = tf.optimizers.Adam(params['lr'])
global_step = 0

history_acc = []
best_acc = .0

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)


while True:
  # TRAINING
  for texts, labels in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      logits = model(texts, training=True)
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
      loss = tf.reduce_mean(loss)
    
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    optim.apply_gradients(zip(grads, model.trainable_variables))

    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1
  
  # EVALUATION
  m = tf.keras.metrics.Accuracy()

  for texts, labels in dataset(is_training=False, params=params):
    logits = model(texts, training=False)
    y_pred = tf.argmax(logits, axis=-1)
    m.update_state(y_true=labels, y_pred=y_pred)
    
  acc = m.result().numpy()
  logger.info("Evaluation: Testing Accuracy: {:.3f}".format(acc))
  history_acc.append(acc)
  
  if acc > best_acc:
    best_acc = acc
    # you can save model here
  logger.info("Best Accuracy: {:.3f}".format(best_acc))
  
  if len(history_acc) > params['num_patience'] and is_descending(history_acc):
    logger.info("Testing Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

W0325 03:30:16.754851 140022398179200 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:421: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


[('kernel_attentive_pooling/dense/kernel:0', TensorShape([300, 1])),
 ('dense_1/kernel:0', TensorShape([300, 2])),
 ('dense_1/bias:0', TensorShape([2])),
 ('pretrained_embedding:0', TensorShape([20599, 300]))]
Reading ../data/train.txt


I0325 03:30:26.536055 140022398179200 interactiveshell.py:2882] Step 0 | Loss: 0.7376 | Spent: 9.8 secs | LR: 0.000300
I0325 03:30:45.361990 140022398179200 interactiveshell.py:2882] Step 50 | Loss: 0.6950 | Spent: 18.8 secs | LR: 0.000299
I0325 03:31:04.888937 140022398179200 interactiveshell.py:2882] Step 100 | Loss: 0.6872 | Spent: 19.5 secs | LR: 0.000298
I0325 03:31:24.072847 140022398179200 interactiveshell.py:2882] Step 150 | Loss: 0.6553 | Spent: 19.2 secs | LR: 0.000298
I0325 03:31:42.505526 140022398179200 interactiveshell.py:2882] Step 200 | Loss: 0.6436 | Spent: 18.4 secs | LR: 0.000297
I0325 03:32:00.057708 140022398179200 interactiveshell.py:2882] Step 250 | Loss: 0.6088 | Spent: 17.6 secs | LR: 0.000296
I0325 03:32:19.260957 140022398179200 interactiveshell.py:2882] Step 300 | Loss: 0.5804 | Spent: 19.2 secs | LR: 0.000295
I0325 03:32:38.410742 140022398179200 interactiveshell.py:2882] Step 350 | Loss: 0.5310 | Spent: 19.1 secs | LR: 0.000295
I0325 03:32:56.727094 140022

Reading ../data/test.txt


I0325 03:35:40.789431 140022398179200 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.867
I0325 03:35:40.791317 140022398179200 interactiveshell.py:2882] Best Accuracy: 0.867


...

I0325 04:09:55.233154 140022398179200 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.889
I0325 04:09:55.234862 140022398179200 interactiveshell.py:2882] Best Accuracy: 0.895


Reading ../data/train.txt


I0325 04:10:22.488271 140022398179200 interactiveshell.py:2882] Step 6300 | Loss: 0.1728 | Spent: 44.8 secs | LR: 0.000217
I0325 04:10:41.382654 140022398179200 interactiveshell.py:2882] Step 6350 | Loss: 0.0873 | Spent: 18.9 secs | LR: 0.000217
I0325 04:11:00.888543 140022398179200 interactiveshell.py:2882] Step 6400 | Loss: 0.0740 | Spent: 19.5 secs | LR: 0.000216
I0325 04:11:18.480540 140022398179200 interactiveshell.py:2882] Step 6450 | Loss: 0.1267 | Spent: 17.6 secs | LR: 0.000215
I0325 04:11:36.405515 140022398179200 interactiveshell.py:2882] Step 6500 | Loss: 0.0763 | Spent: 17.9 secs | LR: 0.000215
I0325 04:11:54.951826 140022398179200 interactiveshell.py:2882] Step 6550 | Loss: 0.0566 | Spent: 18.5 secs | LR: 0.000214
I0325 04:12:13.411118 140022398179200 interactiveshell.py:2882] Step 6600 | Loss: 0.1167 | Spent: 18.5 secs | LR: 0.000214
I0325 04:12:32.519585 140022398179200 interactiveshell.py:2882] Step 6650 | Loss: 0.0745 | Spent: 19.1 secs | LR: 0.000213
I0325 04:12:51.3

Reading ../data/test.txt


I0325 04:15:09.607374 140022398179200 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.887
I0325 04:15:09.609139 140022398179200 interactiveshell.py:2882] Best Accuracy: 0.895
I0325 04:15:09.617720 140022398179200 interactiveshell.py:2882] Testing Accuracy not improved over 3 epochs, Early Stop
